Názorně si předvedeme typické úkony, se kterými se setkáváme při čištění dat oscrapovaných z webu anebo stažených z nepříliš uspořádaného zdroje.

Nejprve je nutné importovat knihovny, které nám s tím pomůžou, v tomto případě **re**, která umožňuje pracovat s tzv. regulárními výrazy (popisují vzorce v textu, jako např. "4 čísla po sobě", [více o nich třeba zde](https://www.regularnivyrazy.info/shrnuti-syntaxe.html)), a **pandas**, nejvyužívanější pythonovskou knihovnu pro práci s daty. (Knihovna **os** zde slouží jen předcházení chyb při práci s adresáři a **statistics** umožní spočítat medián ještě před načtením dat do pandas.)

In [225]:
import os
import re
import statistics
import pandas as pd

Pak si načteme zdrojová data, která se vždy snažíme držet v jiných adresářích než data čistá. Toto bychom viděli, kdybychom tabulku ve formátu CSV otevřeli v texťáku:

![surová data](surova_data.png)

In [227]:
df = pd.read_csv(os.path.join("zdrojova_data","zdrojova_data.csv"))

Jak data vypadají zde?

In [229]:
df

,Albína,1984-07-30,Brno,"45,000"
0,Bedřich,1987-06-15,Brno-Královo Pole,43000
1,Cecílie,jaro 1986,Blansko,45000-48000 (podle měsíce)
2,Dobromila,1994-12-01,Brno,39000


Hned vidíme, že jim chybí záhlaví. Znovu a lépe s header=None:

In [231]:
df = pd.read_csv(os.path.join("zdrojova_data","zdrojova_data.csv"), header=None)

In [232]:
df

,0,1,2,3
0,Albína,1984-07-30,Brno,"45,000"
1,Bedřich,1987-06-15,Brno-Královo Pole,43000
2,Cecílie,jaro 1986,Blansko,45000-48000 (podle měsíce)
3,Dobromila,1994-12-01,Brno,39000


Budeme chtít přejmenovat sloupce:

In [234]:
df = df.rename(columns={0: "jméno", 1: "datum_narození", 2: "místo_narození", 3: "plat"})

In [235]:
df

,jméno,datum_narození,místo_narození,plat
0,Albína,1984-07-30,Brno,"45,000"
1,Bedřich,1987-06-15,Brno-Královo Pole,43000
2,Cecílie,jaro 1986,Blansko,45000-48000 (podle měsíce)
3,Dobromila,1994-12-01,Brno,39000


Jaké je nejčastější místo narození?

In [237]:
df.groupby("místo_narození").size().nlargest()

místo_narození
Brno                 2
Blansko              1
Brno-Královo Pole    1
dtype: int64

Počkat, Královo Pole je přeci v Brně. Takže si vytvoříme nový sloupec, do kterého budeme brát pouze údaje před pomlčkou:

In [239]:
df['místo_narození_čisté'] = df['místo_narození'].apply(lambda x: x.split("-")[0])

Co jsme dostali?

In [240]:
df['místo_narození_čisté']

0       Brno
1       Brno
2    Blansko
3       Brno
Name: místo_narození_čisté, dtype: object

A jak vypadá žebříček teď?

In [241]:
df.groupby("místo_narození_čisté").size().nlargest()

místo_narození_čisté
Brno       3
Blansko    1
dtype: int64

Teď se podíváme, kdo je nejstarší:

In [243]:
df.sort_values(by="datum_narození")

,jméno,datum_narození,místo_narození,plat,místo_narození_čisté
0,Albína,1984-07-30,Brno,"45,000",Brno
1,Bedřich,1987-06-15,Brno-Královo Pole,43000,Brno
3,Dobromila,1994-12-01,Brno,39000,Brno
2,Cecílie,jaro 1986,Blansko,45000-48000 (podle měsíce),Blansko


To "jaro" nám to celé rozhodilo. Zde by stačilo najít rok narození jako 4ciferné číslo a seřadit tabulku podle něj:

In [245]:
df['rok_narození'] = df['datum_narození'].apply(lambda x: re.search(r"\d{4}", x).group())

In [246]:
df['rok_narození']

0    1984
1    1987
2    1986
3    1994
Name: rok_narození, dtype: object

In [247]:
df.sort_values(by='rok_narození')

,jméno,datum_narození,místo_narození,plat,místo_narození_čisté,rok_narození
0,Albína,1984-07-30,Brno,"45,000",Brno,1984
2,Cecílie,jaro 1986,Blansko,45000-48000 (podle měsíce),Blansko,1986
1,Bedřich,1987-06-15,Brno-Královo Pole,43000,Brno,1987
3,Dobromila,1994-12-01,Brno,39000,Brno,1994


Prima! Obvykle je ale nejlepší postup využít metodu pd.to_datetime.

In [252]:
df['datum_narození_čisté'] = pd.to_datetime(df['datum_narození'])

ValueError: time data "jaro 1986" doesn't match format "%Y-%m-%d", at position 2. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

No jo, ale pandas neví, co je to jaro. Chudák pandas.

Musíme tedy metodu upozornit na to, že je formát dat smíšený a že má v případě chyby data zahodit a pokračovat dál:

In [254]:
df['datum_narození_čisté'] = pd.to_datetime(df['datum_narození'], format="mixed", errors="coerce")

In [256]:
df

,jméno,datum_narození,místo_narození,plat,místo_narození_čisté,rok_narození,datum_narození_čisté
0,Albína,1984-07-30,Brno,"45,000",Brno,1984,1984-07-30
1,Bedřich,1987-06-15,Brno-Královo Pole,43000,Brno,1987,1987-06-15
2,Cecílie,jaro 1986,Blansko,45000-48000 (podle měsíce),Blansko,1986,NaT
3,Dobromila,1994-12-01,Brno,39000,Brno,1994,1994-12-01


Tady jsme bohužel přišli o celou informaci v chybném řádku. Někdy můžeme pro další výpočty doplnit vymyšlené datum, někdy si tím koledujeme o průšvih – jsme velcí a musíme se rozhodnout sami:  

In [258]:
df['datum_narození_čisté'] = df['datum_narození_čisté'].fillna(df['rok_narození'])

In [260]:
df

,jméno,datum_narození,místo_narození,plat,místo_narození_čisté,rok_narození,datum_narození_čisté
0,Albína,1984-07-30,Brno,"45,000",Brno,1984,1984-07-30
1,Bedřich,1987-06-15,Brno-Královo Pole,43000,Brno,1987,1987-06-15
2,Cecílie,jaro 1986,Blansko,45000-48000 (podle měsíce),Blansko,1986,1986-01-01
3,Dobromila,1994-12-01,Brno,39000,Brno,1994,1994-12-01


Teď už tedy můžeme spočítat, kolik jim je let:

In [262]:
from datetime import date

today = date.today()

In [264]:
df['věk'] = pd.to_datetime(today)- df['datum_narození_čisté']

In [266]:
df

,jméno,datum_narození,místo_narození,plat,místo_narození_čisté,rok_narození,datum_narození_čisté,věk
0,Albína,1984-07-30,Brno,"45,000",Brno,1984,1984-07-30,14696 days
1,Bedřich,1987-06-15,Brno-Královo Pole,43000,Brno,1987,1987-06-15,13646 days
2,Cecílie,jaro 1986,Blansko,45000-48000 (podle měsíce),Blansko,1986,1986-01-01,14176 days
3,Dobromila,1994-12-01,Brno,39000,Brno,1994,1994-12-01,10920 days


Půjde nám spočítat průměrný plat?

In [268]:
df['plat'].mean()

TypeError: Could not convert string '45,0004300045000-48000 (podle měsíce)39000' to numeric

Nepůjde. Narazili jsme na "TypeError", asi nejčastější problém s nepročištěnými daty: pandas u některých polí neví, jak je interpretovat, jestli jako číslo, nebo jako textový řetězec. Zkusíme základní příkaz pro převod na číselný formát:

In [270]:
df['plat'] = pd.to_numeric(df['plat'])

ValueError: Unable to parse string "45,000" at position 0

Stále je to pro pandas příliš složité, zde se zarazilo hned na čárce v prvním řádku. Když si sloupec prohlédneme celý, vidíme tam zjevných oříšků víc. Všechny jdou ale řešit. Musíme si pro to napsat funkci:

In [272]:
def hezky_plat(udaj_o_platu):
    try: # Nejprve zkusíme vrátit údaj o platu jako celé číslo, to by u 2 ze 4 řádků mělo fungovat
        print(f"Zde převod na celé číslo fungoval bez problémů: {int(udaj_o_platu)}")
        return int(udaj_o_platu) 
    except Exception as E: # A když nám to vrátí chybu, udělám s tím něco jiného:
        if "," in str(udaj_o_platu): # Je v poli čárka? Vyhodíme ji.
            print(f"Vyhodili jsme čárku a funguje to: {int(udaj_o_platu.replace(",",""))}")
            return int(udaj_o_platu.replace(",",""))
        elif "-" in str(udaj_o_platu): ## Největší oříšek je samozřejmě pole s dvěma platy. Co uděláme: najdeme v tomto poli všechna 5ciferná čísla a necháme si vrátit medián.
            peticiferna_cisla = re.findall(r"\d{5}", str(udaj_o_platu))
            peticiferna_cisla = [int(c) for c in peticiferna_cisla]
            print(f"Spočítali jsme medián z tohoto seznamu: {peticiferna_cisla}")
            return statistics.median(peticiferna_cisla)

In [274]:
df['plat_čistý'] = df['plat'].apply(lambda x: hezky_plat(x))

Vyhodili jsme čárku a funguje to: 45000
Zde převod na celé číslo fungoval bez problémů: 43000
Spočítali jsme medián z tohoto seznamu: [45000, 48000]
Zde převod na celé číslo fungoval bez problémů: 39000


Jak vypadá tabulka teď?

In [276]:
df

,jméno,datum_narození,místo_narození,plat,místo_narození_čisté,rok_narození,datum_narození_čisté,věk,plat_čistý
0,Albína,1984-07-30,Brno,"45,000",Brno,1984,1984-07-30,14696 days,45000.0
1,Bedřich,1987-06-15,Brno-Královo Pole,43000,Brno,1987,1987-06-15,13646 days,43000.0
2,Cecílie,jaro 1986,Blansko,45000-48000 (podle měsíce),Blansko,1986,1986-01-01,14176 days,46500.0
3,Dobromila,1994-12-01,Brno,39000,Brno,1994,1994-12-01,10920 days,39000.0


Už můžeme spočítat průměr:

In [278]:
df['plat_čistý'].mean()

43375.0

Pokud bychom tabulku potřebovali k výpočtu důchodu, bude jedno nepřesné datum narození a jeden nepřesný výpočet platu mega průšvih. Ale dejme tomu, že jsem to dělali z jiného důvodu a takto čistá data nám stačí. Nyní si je tedy můžeme uložit pro další práci, a to do jiného adresáře a souboru než originál:

In [280]:
df[['jméno','datum_narození_čisté','místo_narození_čisté','plat_čistý']].to_csv(os.path.join("cista_data","cista_data.csv"))